In [1]:
'''
Application 主架構
'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson = json.load(open('./line_secret_key', 'r', encoding='utf-8'))
server_url = secretFileContentJson.get('server_url')
channel_access_token = secretFileContentJson["channel_access_token"]


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

#生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get('channel_access_token'))

In [2]:
'''
將圖文選單設定檔，post給line，取得rich_menu_id

設定line遠端位置
設定消息的基本安全憑證
寄發消息，並取得rich_menu_id

'''

import requests
import os 
for i in os.listdir('./material/rich_menu'):
    if not os.path.exists('./material/rich_menu/' + i + '/rich_menu_id.json'):
        if i != '.DS_Store':
            menuJson = json.load(open('./material/rich_menu/' + i + '/rich_menu.json','r',encoding='utf-8'))
            createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
            createMenuRequestHeader = {'Content-Type':'application/json','Authorization':'Bearer %s' 
                                        % channel_access_token}
            lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))
            f = open('./material/rich_menu/' + i + '/rich_menu_id.json','w',encoding='utf-8')
            f.write(lineCreateMenuResponse.text)
            f.close()

In [3]:
'''
將圖文選單圖片post給line

讀取rich_menu_id
設定line遠端位置
設定消息的基本安全憑證
上傳照片並取得回傳結果


'''

for i in os.listdir('./material/rich_menu'):
    if i != '.DS_Store':
        # 取得菜單Id         
        rich_menu_id = json.load(open('./material/rich_menu/' + i + '/rich_menu_id.json', 'r', encoding='utf-8'))
        rich_menu_id = rich_menu_id['richMenuId']
        
        # 設定Line的遠端位置        
        uploadMenuEndpoint = 'https://api.line.me/v2/bot/richmenu/%s/content' % rich_menu_id
        
        # 設定消息的基本安全憑證
        uploadMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s'
                                        % channel_access_token}
        
        # 上傳照片
        uploadImageFile = open('./material/rich_menu/' + i + '/' + i + '.jpg', 'rb')
        lineUploadMenuResponse = requests.post(uploadMenuEndpoint, headers=uploadMenuRequestHeader,data=uploadImageFile)
        uploadImageFile.close()
        print(lineUploadMenuResponse)
        print(lineUploadMenuResponse.text)

<Response [400]>
{"message":"An image has already been uploaded to the richmenu"}
<Response [400]>
{"message":"An image has already been uploaded to the richmenu"}
<Response [400]>
{"message":"An image has already been uploaded to the richmenu"}
